# 🌳🕸️ Tree‑ & Graph‑of‑Thought Explorer

**Prompt Engineering – Day 5 Advanced Reasoning Lab**

Dive deep into *structured* multi‑path reasoning:

* **Tree‑of‑Thoughts (ToT)** – branch, evaluate, backtrack  
* **Graph‑of‑Thoughts (GoT)** – flexible DAG exploration & visualization  

This Colab‑ready notebook combines conceptual primers, runnable algorithms, LLM‑powered search stubs (plug‑in your key), and interactive exercises.

---

## 📑 Learning goals

1. Understand the difference between linear Chain‑of‑Thought and branching **Tree/Graph‑of‑Thought** paradigms.  
2. Implement a generic **ToT search loop** with pluggable scoring functions (heuristic or LLM).  
3. Visualize reasoning trajectories with **NetworkX** and **matplotlib**.  
4. Experiment with beam width, depth limits, and reflection‑based pruning.  
5. Apply ToT/GoT to toy puzzles (word scramble, arithmetic planning, path‑finding).

## 🔧 0. Setup

Install deps & set your OpenAI‑compatible key (optional – heuristics will be used if absent).

In [ ]:
%pip -q install --upgrade openai networkx matplotlib tiktoken
import os, openai, random, math, itertools, collections, json, time
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

openai.api_key = os.getenv("OPENAI_API_KEY")  # leave blank to use heuristics only
MODEL = "gpt-4o-mini"


---

## 1️⃣ From Lines to Trees to Graphs

* **Chain‑of‑Thought (CoT)**: single linear rationale  
* **Tree‑of‑Thought (ToT)**: explore *multiple* partial reasoning states, keeping top‑k at each depth  
* **Graph‑of‑Thought (GoT)**: allow merges / loops – states become nodes, transitions become edges

Why? → Better global search, ability to backtrack, parallel exploration, reduced hallucination via cross‑checking paths.

![ToT vs GoT](https://i.imgur.com/7n5bX2e.png)


---

## 2️⃣ Generic ToT Search Framework

We’ll build a reusable `TreeOfThoughtSolver`:

```text
while depth < max_depth:
    expand top‑k states → children
    score children (heuristic or LLM)
    keep best beam_width states
    optional: reflection/pruning
```

States and expansion logic are **task‑specific** – we’ll provide two examples.

In [ ]:
class TreeOfThoughtSolver:
    def __init__(self, expand_fn, score_fn, beam_width=5, max_depth=5, verbose=True):
        """Generic beam/breadth search."""
        self.expand_fn = expand_fn
        self.score_fn = score_fn
        self.beam_width = beam_width
        self.max_depth = max_depth
        self.verbose = verbose

    def solve(self, init_state):
        frontier = [(init_state, 0.0)]  # (state, score)
        for depth in range(self.max_depth):
            if self.verbose:
                print(f"\nDepth {depth} – frontier size {len(frontier)}")
            candidates = []
            for state, _ in frontier:
                children = self.expand_fn(state)
                for c in children:
                    score = self.score_fn(c)
                    candidates.append((c, score))
            # keep top beam_width
            frontier = sorted(candidates, key=lambda x: -x[1])[:self.beam_width]
            # check goal
            for s, sc in frontier:
                if isinstance(sc, tuple) and sc[1]:  # (score, is_goal)
                    return s
        return frontier[0][0]  # best effort


### 🧩 Example A – Word Scramble

Goal: unscramble 5‑letter word (like 'TRAIN' → 'RIANT', 'RATIN', 'NAIRT', etc.) that exists in English word list.

In [ ]:
import requests, itertools, random, string
WORDS = {w.strip().lower() for w in requests.get('https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt').text.splitlines() if len(w)==5}

def expand_scramble(state):
    prefix, remaining = state
    if not remaining:
        return []
    children=[]
    for i,ch in enumerate(remaining):
        new_prefix = prefix+ch
        new_remaining = remaining[:i]+remaining[i+1:]
        children.append((new_prefix, new_remaining))
    return children

def score_scramble(state):
    prefix, remaining = state
    if not remaining and prefix.lower() in WORDS:
        return (1.0, True)  # perfect & goal
    return (len(prefix)/5, False)

scramble = "TRAIN"
init_state = ("", scramble)
solver = TreeOfThoughtSolver(expand_scramble, score_scramble, beam_width=10, max_depth=5)
result = solver.solve(init_state)
print("Best result:", result[0])

#### 📝 Try‑it

Change `scramble` to other 5‑letter jumbles.  
Play with `beam_width` and `max_depth`.  
Why might beam search sometimes miss a solution?

### ➕ Example B – Arithmetic Planning via LLM Scoring

Task: reach target number using given digits and `+ − × ÷`.  
We’ll let the *LLM* judge how close an expression is to the target (quick demonstration – not guaranteed optimal!).

In [ ]:
target = 42
digits = [3,7,9]
ops = ['+','-','*','/']

def expand_arith(expr):
    if len(expr.split())//2 >= len(digits)-1:
        return []
    next_digit = digits[len(expr.split())//2 + 1]
    children=[]
    for op in ops:
        children.append(expr + f" {op} {next_digit}")
    return children

def llm_score(expr):
    if not openai.api_key:
        # fallback heuristic: inverse absolute error
        try:
            val = eval(expr)
            return -abs(target-val)
        except ZeroDivisionError:
            return -999
    prompt = f"Evaluate how close the value of the expression '{expr}' is to {target}. Respond with a single float: negative absolute error (higher is better)." 
    try:
        reply = chat("You are a grading function.", prompt, temperature=0)
        return float(reply)
    except:
        return -999

init_expr = str(digits[0])
solver2 = TreeOfThoughtSolver(expand_arith, llm_score, beam_width=4, max_depth=3)
best = solver2.solve(init_expr)
print("Best expression:", best)

---

## 3️⃣ Graph‑of‑Thoughts Visualizer

Convert explored states into a directed graph to see *all* reasoning paths, branch merges, and dead‑ends.

In [ ]:
def explore_graph(init_state, expand_fn, depth=3):
    G=nx.DiGraph()
    G.add_node(str(init_state))
    frontier=[init_state]
    for _ in range(depth):
        new=[]
        for st in frontier:
            for ch in expand_fn(st):
                G.add_edge(str(st), str(ch))
                new.append(ch)
        frontier=new
    return G

G = explore_graph(init_state, expand_scramble, depth=3)
plt.figure(figsize=(10,6))
nx.draw(G, with_labels=False, node_size=150, arrows=False)
plt.title("Graph‑of‑Thought (partial scramble search)")
plt.show()

#### 📝 Exercise

* Replace `expand_scramble` with your own expansion logic (e.g., arithmetic plan).  
* Use `nx.drawing.nx_pydot.write_dot(G, 'graph.dot')` to export for GraphViz.

---

## 4️⃣ Reflection‑Based Pruning (Bonus)

Many ToT papers add a **reflection** step: ask the LLM to assess partial states and prune unlikely branches early.  
Implement a simple demo (stubbed).

In [ ]:
def reflect_and_score(state):
    prefix, remaining = state
    if not openai.api_key:
        # cheap heuristic
        return (len(prefix)/5, False)
    prompt = f"You are judging word prefixes. Does '{prefix}' look like a valid English word prefix? Reply 1 for good, 0 for bad."
    ok = chat("Judge", prompt, temperature=0)
    good = float(ok.strip() or 0)
    return (good, False)

solver3 = TreeOfThoughtSolver(expand_scramble, reflect_and_score, beam_width=5, max_depth=5)
best = solver3.solve(init_state)
print(best)

---

## 5️⃣ Capstone Challenge

1. Pick **any** problem domain (logic puzzle, Sudoku, route planning).  
2. Define state representation + `expand_fn` + `score_fn` *(heuristic or LLM)*.  
3. Use `TreeOfThoughtSolver` to search for a solution.  
4. Build a **Graph‑of‑Thought** for your search and visualize.  
5. Reflect: where did branching help vs. hinder?

---

## 🔗 Key Papers

* Yao et al., “Tree‑of‑Thought: Deliberate reasoning via expansive tree search”, 2023  
* Long & Bosch, “Graph‑of‑Thought: Solving Elaborate Problems via Structured Memory”, 2024  
* Gao et al., “Plan-and-Reflect: Leveraging Self‑Critique for Better ToT Search”, 2024